# Crime Scene Investigation

## Fighting un-Pythonic Crime

### ByteCode At a Time

#### Let [PEP-8](https://www.python.org/dev/peps/pep-0008/) Be With You!

In [ ]:
import dis
dis.__doc__

In [ ]:
dis.dis('x = 1')

In [1]:
import dis
import io
import itertools
import re
import sys

class IoCatcher:
    def __init__(self):
        self.old_stdout = sys.stdout
        sys.stdout = self.buffer = io.StringIO()
    def __enter__(self):
        return self.buffer
    def __exit__(self, *_):
        sys.stdout = self.old_stdout

def comparative_dis(*snippets):
    compressor = re.compile(r'( +)'
                             '(  | \d|\d\d)' # 2
                             '( +)'
                             '(  |>>)' # 4
                             '( +)'
                             '([ \d]\d +\w+)' #6
                             '( {4})?'
                           )    
    snippets_dis = []
    for snippet in snippets:
        with IoCatcher() as io_catcher:
            dis.dis(snippet)
        snippets_dis.append(list(map(lambda l: compressor.sub(r' \2 \4 \6', l),
                                     io_catcher.getvalue().splitlines())))
    widths = [max(map(len, snippet)) for snippet in snippets_dis]
    fmt = ' | '.join('{{:<{}}}'.format(w) for w in widths)
    print(*(fmt.format(*lines) for lines in itertools.zip_longest(*snippets_dis, fillvalue='')), 
          sep='\n')

## Truthiness of Sequences

> For sequences, (strings, lists, tuples), use the fact that empty sequences are false:

```python
# Correct:
if not seq:
if seq:

# Wrong:
if len(seq):
if not len(seq):

```

### The Proof

In [ ]:
comparative_dis('if seq: pass', 'if len(seq): pass')

In [ ]:
comparative_dis('if not seq: pass', 'if not len(seq): pass')

## Booleans - again

> Don't compare boolean values to _True_ or _False_ using __==__:

```python
# Correct:
if greeting:

# Wrong:
if greeting == True:

# Worse:
if greeting is True:
```

### The Proof

In [ ]:
comparative_dis('if flag: pass', 'if flag == True: pass', 'if flag is True: pass')

## Type Check
> Object type comparisons should always use isinstance() instead of comparing types directly:

```python
# Correct:
if isinstance(obj, int):

# Wrong:
if type(obj) is int:

# Worst
if type(obj) == int:
```

### The Proof

In [ ]:
comparative_dis('if isinstance(obj, int): pass', 'if type(obj) is int: pass', 'if type(obj) == int: pass')

# So far, PEP-8 is partially covered. What next?

## Key in a Dictionary

```python
# Correct 
if 'key' in some_dict:
for key in some_dict:
    
# Wrong
if 'key' in some_dict.keys()
for key in some_dict.keys():
```


### The Proof

In [ ]:
comparative_dis('"key" in some_dict', '"key" in some_dict.keys()')

In [ ]:
comparative_dis('for key in some_dict: pass', 'for key in some_dict.keys(): pass')

## Iteration Over List

In [ ]:
def sum_list_correct(arr):
    list_sum = 0
    for elem in arr:
        list_sum += elem
    return list_sum

def sum_list_incorrect(arr):
    list_sum = 0
    for idx in range(len(arr)):
        list_sum += arr[idx]
    return list_sum

### The Proof

In [ ]:
comparative_dis(sum_list_correct, sum_list_incorrect)